<a href="https://colab.research.google.com/github/isabellaloren4/isabellaloren4/blob/main/modlamp_features_acp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install modlamp
!pip install numpy
!pip install rdkit-pypi

In [ ]:
from multiprocessing import allow_connection_pickling
#Sequências do ACP2.0
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from modlamp.descriptors import PeptideDescriptor

In [ ]:
facp = open("acp.txt",'r')
fnacp = open("nacp.txt", 'r')

acp = facp.readlines()
nacp = fnacp.readlines()

facp.close()
fnacp.close()

tam_acp=len(acp)
tam_nacp=len(nacp)

#Criando a lista para sequências em FASTA
fasta=[]
y=[]

for i in range(tam_acp):
    fasta.append((acp[i][:-1]))
    y.append(1)

for i in range(tam_nacp):
    fasta.append((nacp[i][:-1]))
    y.append(0)

#Convertendo fasta em objeto
fasta=np.array(fasta)

In [ ]:
#Peptide descriptor

descr_names = ['AASI', 'ABHPRK', 'argos', 'bulkiness', 'charge_phys', 'charge_acid', 'cougar',
               'eisenberg', 'Ez', 'flexibility', 'grantham', 'gravy', 'hopp-woods', 'ISAECI',
               'janin', 'kytedoolittle', 'levitt_alpha', 'MSS', 'MSW', 'pepArc', 'pepcats',
               'polarity', 'PPCALI', 'refractivity', 't_scale', 'TM_tend', 'z3', 'z5']

D = []

for descr in descr_names:

    #calcular descritor autocorr
    #d = PeptideDescriptor(fasta, scalename=descr)
    #d.calculate_crosscorr(2)
    #dc = d.descriptor

    #calcular descritor
    pepdesc = PeptideDescriptor(fasta, scalename=descr)
    pepdesc.calculate_global()
    dc = pepdesc.descriptor

    #converter para numpy array corretamente
    dc_numpy = np.zeros((len(dc), len(dc[0])))
    for i in range (len(dc)):
        dc_numpy[i] = np.array(dc[i])

    D.append(dc_numpy)

In [ ]:
M = np.concatenate((D[0], D[1]), axis=1)

for i in range(2,len(D)):
    M = np.concatenate((M, D[i]), axis=1)

In [ ]:
pd.DataFrame(M).to_csv('features/modlamp.csv', index=False, header=False)